In [1]:
import mxnet as mx
from mxnet.gluon import nn
from mxnet import nd

In [2]:
# 使用 NDArray 实现一个 Gluon 层
from mxnet import gluon
my_param = gluon.Parameter('my parameter', shape=(3, 3))

In [4]:
my_param.initialize()
print(my_param.data())
my_param.grad()


[[ 0.0068339   0.01299825  0.0301265 ]
 [ 0.04819721  0.01438687  0.05011239]
 [ 0.00628365  0.04861524 -0.01068833]]
<NDArray 3x3 @cpu(0)>


/Users/poodarchu/anaconda3/lib/python3.6/site-packages/mxnet/gluon/parameter.py:320: UserWarning: Parameter my parameter is already initialized, ignoring. Set force_reinit=True to re-initialize.
  "Set force_reinit=True to re-initialize."%self.name)



[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
<NDArray 3x3 @cpu(0)>

In [5]:
# 自定义层时 通常使用 nn.Block 自带的一个 ParameterDict 类型的成员变量 params
pd = gluon.ParameterDict(prefix='block1_')
pd.get('my_parameter1', shape=(3, 3))

Parameter block1_my_parameter1 (shape=(3, 3), dtype=<class 'numpy.float32'>)

In [6]:
pd

block1_ (
  Parameter block1_my_parameter1 (shape=(3, 3), dtype=<class 'numpy.float32'>)
)

In [9]:
class MyDense(nn.Block):
    def __init__(self, units, in_units, **kwargs):
        super(MyDense, self).__init__(**kwargs)
        with self.name_scope():
            self.weight = self.params.get('weight', shape=(in_units, units))
            self.bias = self.params.get('bias', shape=(units,))
    def forward(self, x):
        linear = nd.dot(x, self.weight.data()) + self.bias.data()
        return nd.relu(linear)

In [11]:
dense = MyDense(5, in_units=10, prefix='o_my_dense_')
dense.params

o_my_dense_ (
  Parameter o_my_dense_weight (shape=(10, 5), dtype=<class 'numpy.float32'>)
  Parameter o_my_dense_bias (shape=(5,), dtype=<class 'numpy.float32'>)
)

In [12]:
dense.initialize()
dense(nd.random.uniform(shape=(100, 10)))


[[ 0.09419049  0.06561558  0.01997661  0.00538119  0.04200276]
 [ 0.12424787  0.03711477  0.12840702  0.          0.        ]
 [ 0.10072421  0.02880572  0.05866724  0.          0.        ]
 [ 0.13373846  0.          0.07931243  0.          0.00669018]
 [ 0.15610188  0.          0.1225276   0.          0.        ]
 [ 0.10177656  0.0224059   0.07291803  0.          0.        ]
 [ 0.11053525  0.02454864  0.0446313   0.          0.03921533]
 [ 0.08504533  0.03005803  0.00178412  0.          0.        ]
 [ 0.16260403  0.0829415   0.08493532  0.          0.00302853]
 [ 0.04783937  0.00142642  0.07829061  0.          0.        ]
 [ 0.1432682   0.09585547  0.10055898  0.          0.        ]
 [ 0.10601809  0.          0.11833167  0.00430752  0.        ]
 [ 0.1308268   0.          0.084801    0.          0.01773058]
 [ 0.13251811  0.          0.08715193  0.          0.        ]
 [ 0.12730613  0.08133221  0.06403394  0.          0.        ]
 [ 0.15895674  0.07628597  0.09426175  0.          0.0

In [13]:
net = nn.Sequential()
with net.name_scope():
    net.add(MyDense(32, in_units=64))
    net.add(MyDense(2, in_units=32))
net.initialize()
net(nd.random.uniform(shape=(2,64)))


[[ 0.          0.07717714]
 [ 0.          0.07592615]]
<NDArray 2x2 @cpu(0)>